# 모델링

train 데이터를 가져와서 모델링에 필요한 **Ordered Target Encoding**을 시행한다.

In [1]:
import pandas as pd
import numpy as np
import os
#get current path
currentPath = os.getcwd()

#change path
print(currentPath)

C:\Users\Lenovo\Desktop\P-Sat\20-1 방세\완성본


In [2]:
dat = pd.read_csv('train.csv')

In [3]:
dat.head()

click           event_datetime      ssp_id campaign_id    adset_id  \
0      0  2019-10-01 00:00:04.878  A6E0SZLhXP  taRA9jVfVL  GdBSlETcLy   
1      0  2019-10-01 00:00:04.940  CD3hRiI3bN  jWRKzxzhyX  GlheP2trvZ   
2      0  2019-10-01 00:00:04.963  SrN77Arvqh  DW5C3As8ij  WGJnvetv2a   
3      1  2019-10-01 00:00:05.186  nwf1A3O5cO  qR4Xa60DLl  FiSRHSfVaf   
4      0  2019-10-01 00:00:05.289  SrN77Arvqh  2T5sOm2MoW  UASfSkWw7S   

  placement_type    media_id  media_name media_bundle media_domain  ...  \
0     1pcQ3RJgQt  IyDyyhXBnW  dAWR8DOmzo   V8yCzbCKcB   pjBT3sDGbH  ...   
1     1pcQ3RJgQt  hkFCnTpDpn  Xbdchs5uK3   o9tQ73S7C8   pjBT3sDGbH  ...   
2     1pcQ3RJgQt  hkFCnTpDpn  Xbdchs5uK3   p75piF74nC   pjBT3sDGbH  ...   
3     kIeE1J0KCa  j7H2fWftrL  Uk5MGt9vxz   DldwurRI4R   pjBT3sDGbH  ...   
4     1pcQ3RJgQt  hkFCnTpDpn  Xbdchs5uK3   7jwRHIrTWJ   pjBT3sDGbH  ...   

  device_carrier device_make device_connection_type device_language  \
0     wrzW2uRaKh  HiGuczmB3W             Hx7e3tE5mu      8GSkINt29e   
1     wrzW2uRaKh  rp6gWKT7zk             WCK2G73H3A      LXYiFbLmMp   
2     wrzW2uRaKh  rp6gWKT7zk             Hx7e3tE5mu      8GSkINt29e   
3     PH0EDk0nOb  javoIwtGqV             WCK2G73H3A      LXYiFbLmMp   
4     wrzW2uRaKh  rp6gWKT7zk             Hx7e3tE5mu      8GSkINt29e   

  device_region device_city advertisement_id  weekday sin_time cos_time  
0    EdsKnCCtPO  e7JXTJQ5Qw       TbkcVoisoR  Tuesday      0.0      1.0  
1    aI9W3Httpm  uuYScbMuRa       Hy6SSlFrrj  Tuesday      0.0      1.0  
2    EdsKnCCtPO  e7JXTJQ5Qw       7dyzy9aZoJ  Tuesday      0.0      1.0  
3    EdsKnCCtPO  e7JXTJQ5Qw       7dyzy9aZoJ  Tuesday      0.0      1.0  
4    EdsKnCCtPO  e7JXTJQ5Qw       hEbfPBEL3I  Tuesday      0.0      1.0  

[5 rows x 26 columns]

In [4]:
dat_x = dat.drop(['click', 'event_datetime'], axis = 1)
dat_y = dat.loc[:, ['click']]

In [5]:
from category_encoders.cat_boost import CatBoostEncoder

def catboost_encoder_notest(train_x, train_y):
  cate_var = list(train_x.select_dtypes(include=['object', 'int64']))
  encoder = CatBoostEncoder()
  cbe_train_x = encoder.fit_transform(train_x[cate_var], train_y)
  cbe_train_result = np.concatenate((cbe_train_x, train_x.loc[:,['sin_time', 'cos_time']]), axis=1)
  cbe_final_train = pd.DataFrame(cbe_train_result, columns= train_x.columns)
  return(cbe_final_train)

# 캣부스트 인코딩을 위한 함수, test셋이 없을 경우 사용하는 함수

In [7]:
train_cbe_x = catboost_encoder_notest(dat_x, dat_y)

파라미터 튜닝은 1,2,3,4,5일 중 5일을 validation set으로 지정해 튜닝했고, 최적 파라미터 결과값으로 모델을 만든다.

## 1. Logistic regression with L2 penalty

로지스틱 회귀분석은 sparse한 범주들을 다룰때 문제가 생긴다. 따라서 1)인코딩과 2)L2페널티로 이를 완화해주려 한다. 

L1 페널티보다는 더 빠른 속도를 기대하고, 예측만을 위하기 때문에 L2 페널티 사용했다.

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss

In [9]:
log_cbe_clf = LogisticRegression(penalty = 'l2', C = 0.9, max_iter = 400)
log_cbe_clf.fit(train_cbe_x, dat_y)

# 파라미터 튜닝했을 때 C = 0.9일때 Validation Logloss가 0.21618 나옴

C:\Users\Lenovo\anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


LogisticRegression(C=0.9, max_iter=400)

In [10]:
import pickle
import joblib

# 모델 저장을 위해 가져온다.

In [11]:
joblib.dump(log_cbe_clf, 'logistic.pkl')
# 모델 저장

['logistic.pkl']

## 2. Light GBM

In [12]:
import lightgbm as lgb
from lightgbm import LGBMModel,LGBMClassifier

In [13]:
best_params = {'max_depth': [10],
               'learning_rate': [0.001],
               'bagging_fraction' : [0.1],
               'max_bin' : [100]}

# 최적 파라미터에서 validation logloss 0.21015

In [14]:
lgbm = LGBMClassifier(**best_params)
lgbm.fit(train_cbe_x, dat_y.to_numpy().flatten(),  eval_metric='logloss',
         verbose=True)

LGBMClassifier(bagging_fraction=[0.1], learning_rate=[0.001], max_bin=[100],
               max_depth=[10])

In [15]:
joblib.dump(lgbm, 'lgbm.pkl')
# 모델 저장

['lgbm.pkl']

## 3. Catboost

In [16]:
from catboost import CatBoostClassifier, Pool

In [17]:
cat_features = list(dat_x.select_dtypes(include=['object', 'int64']))
# 범주형 변수 지정

catboost_clf = CatBoostClassifier(iterations = 106,
                                  learning_rate = 0.15,
                                  loss_function = 'Logloss',
                                  eval_metric = 'Logloss')

# 최적 파라미터에서 validation logloss 0.21065

In [18]:
catboost_clf.fit(dat_x, dat_y,
                 cat_features = cat_features,
                 use_best_model = False,
                 verbose = 50)

0:	learn: 0.5210810	total: 2.71s	remaining: 4m 44s
50:	learn: 0.2313593	total: 1m 35s	remaining: 1m 43s
100:	learn: 0.2301384	total: 3m 17s	remaining: 9.76s
105:	learn: 0.2299431	total: 3m 27s	remaining: 0us


In [19]:
catboost_clf.save_model('catboost_clf')

# 모델 저장

### 이상 모델링 끝!

logistic, lgbm, catboost 셋 다 완료했습니다.

전처리부터 모델링까지 12분 이내

CPU : 인텔 8세대 I5, RAM : 8GB, GPU : X